## Language Model
- 문장의 확률을 나타낸 모델
  - 문장 자체의 출현 확률 or 이전 단어가 주어졌을 때 다음 단어를 예측
- 상황 feature를 추가?
  - 언어에서 상황을 유추해서 반영?
  - 상황이 주어졌을 때 언어 모델에 영향
  - 상황에 대한 범주화가 가능한가?
- 목표는 우리가 일상에 사용하는 언어의 문장 분포를 정확하게 모델링 하는 것

### Korean is Hell?
- 단어의 어순이 중요하지 않음
  - 한국어 문장은 여러개의 소문장으로 구성?
  - 그러면 FastText와 같은 방식 이용이 가능할까? (subsentence segmentation?)
- 단어의 생략이 가능

### Chain Rule
- P(A,B,C,D) = P(D|A,B,C)P(A,B,C) = P(D|A,B,C)P(C|A,B)P(A,B) = P(D|A,B,C)P(C|A,B)P(B|A)P(A)
- P(x_1:n) = P(x1, x2, ..., xn) = cumprod( P(x_i|x_\<i) )
- log P(x_1:n) = sigma( log P(x_i | x_<i) )
- log P(x_1:n ; theta) = sigma( log P(x_j | x_<j ; theta))
- argmax sigma( log P(x_1:n) ) = 위에서 변환

## Count based Approximation
- P(x_n | x_<n) := COUNT(x_1, x_2, ... , x_n) / COUNT(x_1, x_2, ... , x_n-1)
- word sequence가 없다면?
  - P(D|A,B,C)P(C|A,B)P(B|A)P(A) <- 어느 하나라도 0이 되는순간 확률이 사라짐

### Markov Assumption
- 직전의 k개의 token만 확인해서 근사
- P(x_n | x_<n) := P(x_n | x_n-1, ... x_n-k) := COUNT(x_n-k, ..., x_n) / COUNT(x_n-k, ..., x_n-1)

### n-gram
- n = k + 1
- k = 0, 1-gram = uni-gram / k = 1, 2-gram = bi-gram / k = 2, 3-gram = tri-gram
- n이 커질수록 확률이 정확하게 표현되기 어려움 -> tri-gram을 가장 많이 사용
#### SRILM
- ngram-count: LM을 훈련
- ngram: LM을 활용

### Smoothing & Discounting
- Markov assumtion을 써도 training corpus에 없다면?
- unseen word sequence에 대한 대처
  - Smoothing, Discounting
- Add One Smoothing
  - P(w_t | w_<t) := C(w_1:t) / C(w_1:t-1) := [ C(w_1:t) + 1 ] / [ C(W_t-1) + |V| ]
  - := [ C(w_1:t) + m * P(w_t) ] / [ C(w_1:t-1) + m ]
- Kneser-Ney Discounting
  - C(learning) > C(laptop), 딥러닝 수업이기 때문에 learning 빈도가 높음
  - Deep learning, machine learning은 deep, machine과 자주 나오지만
  - laptop은 다양한 단어 뒤에서 나타나지 않았을까?
  - 그렇다면 다양한 단어 뒤에서 나타나는 단어일수록 unseen word sequence에 등장 확률이 높을것이다.
  - laptop보다는 learning이 전체 빈도는 적지만, 등장 확률이 높을 수 있다.

### Interpolation and Back-off
- Interpolation (보간법)
  - 다른 Language model을 linear하게 일정 비율 섞는것
  - general LM + domain specific LM
  - P(w_n|w_nk...w_n-1) = kP_1(w_n| ...) + (1-k)P_2(w_n| ...)
  - 단순히 corpus를 합쳐서 만들경우, domain specific corpus가 너무 sparse해서 반영 안될수도 있음
- Back-off
  - uni-gram, bi-gram, tri-gram으로 만들어진 문장을 interpolation
  - OoV가 없다면 언젠가는 확률이 구해짐


## Perplexity, evaluate LM
- 좋은모델 - 실제 사용하는 언어의 분포를 가장 잘 근사한 모델
- -> 잘 정의된 테스트셋의 문장에 대해 높은 확률을 반환하는 언어모델
- PPL
  - 테스트 문장에 대해 확률을 구하고, PPL 수식에 넣어 성능 측정
  - PPL(x_1, ..., x_n;theta) = P(x_1, ..., x_n;theta) ** (-1/n)
  - 문장의 확률을 길이에 대해 normalization(기하평균)
  - time-step 별 평균 branch의 수 = 매 스텝에서 평균적으로 헷갈리고 있는 단어 수
    - PPL이 낮다 = 확률 분포가 sharp
    - PPL이 높다 = 확률 분포가 Flat